In [6]:
%serialconnect

Connecting to Serial /dev/ttyUSB1 baud=115200

 ** Serial connected **

Serial<id=0x7f8ef7b3f160, open=True>(port='/dev/ttyUSB1', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)


*** Sending Ctrl-C

ets Jun  8 2016 00:22:57

rst:0x1 (POWERON_RESET),boot:0x13 (SPI_FAST_FLASH_BOOT)
flash read err, 1000
Falling back to built-in command interpreter.
OK
>ets Jun  8 2016 00:22:57

rst:0x10 (RTCWDT_RTC_RESET),boot:0x13 (SPI_FAST_FLASH_BOOT)
configsip: 0, SPIWP:0xee
clk_drv:0x00,q_drv:0x00,d_drv:0x00,cs0_drv:0x00,hd_drv:0x00,wp_drv:0x00
mode:DIO, clock div:2
load:0x3fff0010,len:4
load:0x3fff0014,len:4268
load:0x40078000,len:0
load:0x40078000,len:10648
entry 0x4007a56c
OSError: [Errno 2] ENOENT
MicroPython v1.9.2-273-g56f18602 on 2017-09-25; ESP32 module with ESP32
Type "help()" for more information.
>>>[reboot detected 0]
>>> 
>>> 
>>> 
>>> 
raw REPL; CTRL-B to exit
> 

In [7]:
print("hi there")
asd

hi there
Traceback (most recent call last):
  File "<stdin>", line 2, in <module>
NameError: name 'asd' is not defined


In [8]:
%lsmagic

usage: %serialconnect [--raw] [portname] [baudrate]
%lsmagic list magic commands
%suppressendcode doesn't send x04 or wait to read after sending the cell
  (assists for debugging using %writebytes and %readbytes)
%writebytes does serial.write() on a string b'binary_stuff' 
usage: %writebytes [-b] stringtosend
%readbytes does serial.read_all()
%rebootdevice reboots device
%disconnects disconnects serial
usage: %sendtofile [-a] destinationfilename
usage: %socketconnect [--raw] ipnumber portnumber


In [2]:
import network, time, socket

si = network.WLAN(network.STA_IF)
si.active(True)
#assert not si.isconnected()

knownwifis = dict(k.split(b":")[:2]  for k in open("wificodes.txt", "rb"))
wifiname = max((sc  for sc in si.scan()  if sc[0] in knownwifis), key=lambda X: X[3])[0]  # X[3]=RSSI (received signal strength)
print("strongest known DoESWifi signal", wifiname)
si.connect(wifiname, knownwifis[wifiname])
del knownwifis

while not si.isconnected():
    time.sleep(0.1)
    pass

ipnumber = si.ifconfig()[0]
print("Connect through wifi to", ipnumber)


strongest known DoESWifi signal b'DoESLiverpool'
> auth (b0)
> assoc (0)
> run (10)
.
Connect through wifi to 192.168.0.29


In [9]:
assert si.isconnected()

s = socket.socket()
portnumber = 9993

s.bind(socket.getaddrinfo("0.0.0.0", portnumber)[0][-1])  # recommended to always make addresses via getaddrinfo()
s.listen(1)  # no backlog of connections allowed

print("\nPlease connect to ({}, {})".format(ipnumber, portnumber))
print("mac-address {}".format(":".join(hex(x)[-2:]  for x in si.config("mac"))))
espname = "ESP_{}".format("".join(hex(x)[-2:]  for x in si.config("mac")[-3:]))


[leftinbuffer] ['ets Jun  8 2016 00:22:57']
[leftinbuffer] ['']
[leftinbuffer] ['rst:0x1 (POWERON_RESET),boot:0x17 (SPI_FAST_FLASH_BOOT)']
[leftinbuffer] ['flash read err, 1000']
[leftinbuffer] ['Falling back to built-in command interpreter.']
[leftinbuffer] ['OK']
[leftinbuffer] ['>ets Jun  8 2016 00:22:57']
[leftinbuffer] ['']
[leftinbuffer] ['rst:0x10 (RTCWDT_RTC_RESET),boot:0x17 (SPI_FAST_FLASH_BOOT)']
[leftinbuffer] ['configsip: 0, SPIWP:0xee']
[leftinbuffer] ['clk_drv:0x00,q_drv:0x00,d_drv:0x00,cs0_drv:0x00,hd_drv:0x00,wp_drv:0x00']
[leftinbuffer] ['mode:DIO, clock div:2']
[leftinbuffer] ['load:0x3fff0010,len:4']
[leftinbuffer] ['load:0x3fff0014,len:4268']
[leftinbuffer] ['load:0x40078000,len:0']
[leftinbuffer] ['load:0x40078000,len:10648']
[leftinbuffer] ['entry 0x4007a56c']
[leftinbuffer] ['\x1bI (982) cpu_start: Pro cpu up.\x1b']
[leftinbuffer] ['\x1bI (982) cpu_start: Single core mode\x1b']
[leftinbuffer] ['\x1bI (984) heap_init: Initializing. RAM available for dynamic alloca

In [7]:
print("Waiting for a connection on ({}, {})".format(ipnumber, portnumber))
ss, addr = s.accept()
print("connection made incoming from", addr)


Waiting for a connection on (192.168.0.29, 9993)
..........connection made incoming from ('192.168.0.215', 51600)


In [9]:
ss.settimeout(0.2)
buf = []
bufsize = 0

def execcc(buf, bufsize):
    if bufsize > 50:
        return b"We will deal big things later\r\n"
    l = (b"".join(buf).decode().strip()).splitlines()
    if len(l) != 1:
        return b"We don't eval multiple lines {}\r\n".format(len(l))
    try:
        res = str(eval(l[0]))
    except Exception as e:
        res = str(e)
    return b"The answer is: {}\r\n".format(res.encode())



In [67]:
# process incoming signals from the main jupyter syste,
k = None
while True:
    try:
        if k is None:
            k = ss.recv(100)
            print("read", str(k))
    except OSError as e:
        if e.args[0] == 110: # ETIMEDOUT
            print(":", end="")
            continue
        raise
    k = k.split(b"\x04", 1)
    buf.append(k[0])
    bufsize += len(k[0])
    if len(k) == 1:
        k = None
        continue
    k = k[1]
    ss.write(b"OK")
    ss.write(b"Thank you for sending {} characters\r\n".format(bufsize))
    ss.write(execcc(buf, bufsize))
    ss.write(b"\x04\x04>")
    buf.clear()
    bufsize = 0
    

Traceback (most recent call last):
  File "<stdin>", line 12, in <module>
  File "<stdin>", line 6, in <module>
NameError: name 'ss' is not defined


In [4]:
print(1)
print(ss.readline())


1
Traceback (most recent call last):
  File "<stdin>", line 2, in <module>
NameError: name 'ss' is not defined


In [57]:
ss.close()

In [8]:
print(ss.recv(100))


b'thiddng'


In [19]:
ss.write(b"OK\x04\04> ")

entering workingserialreadallentering workingserialreadallentering workingserialreadall

In [18]:
ss.write(b" ")

entering workingserialreadallentering workingserialreadallentering workingserialreadall

In [6]:
%disconnect

Closing serial Serial<id=0x7fd3d8b25a20, open=True>(port='/dev/ttyUSB2', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)


In [3]:
sdfsdf

Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
NameError: name 'sdfsdf' is not defined


In [62]:
%sendtofile wificodes.txt

DoESLiverpool:------:
DoESBackup:------:
The Sensible Code Company:------:

3 lines sent done

In [4]:
for i in range(10):
    print(4**i)

1
4
16
64
256
1024
4096
16384
65536
262144


In [1]:
%lsmagic

%disconnect
    disconnects serial

%lsmagic
    list magic commands

%readbytes
    does serial.read_all()

%rebootdevice
    reboots device

%sendtofile [-a] destinationfilename
    send cell contents or file to device file

%serialconnect [--raw] [portname] [baudrate]
    connects to a device over USB wire

%socketconnect [--raw] ipnumber portnumber
    connects to a socket of a device over wifi

%suppressendcode
    doesn't send x04 or wait to read after sending the cell
  (assists for debugging using %writebytes and %readbytes)

%writebytes [-b] stringtosend
    does serial.write() of the python quoted string given

